In [71]:
import polars as pl

# Scan the big parquet file and tot up the proportions of the winners
df_counts = (
    pl.scan_parquet("results.parquet")
    .group_by("monkey_strategy", "wolf_strategy")
    .agg(pl.col("winner").value_counts(normalize=True))
    .explode("winner")
    .unnest("winner")
    .collect()
)
df_counts

monkey_strategy,wolf_strategy,winner,proportion
str,str,str,f64
"""metric_strength""","""metric_count""","""monkey""",0.579
"""metric_strength""","""metric_count""","""wolf""",0.421
"""metric_position""","""metric_count""","""monkey""",0.309
"""metric_position""","""metric_count""","""wolf""",0.691
"""random""","""metric_strength""","""wolf""",0.956
…,…,…,…
"""random_spot_win""","""metric_position""","""monkey""",0.3
"""random_spot_win""","""random_spot_win""","""monkey""",0.562
"""random_spot_win""","""random_spot_win""","""wolf""",0.438


In [73]:
strategies = pl.concat(df_counts.select("monkey_strategy", "wolf_strategy")).unique().alias("strategy")
strategies

strategy
str
"""random_spot_win"""
"""random"""
"""metric_position"""
"""metric_count"""
"""metric_strength"""
